In [79]:
from __future__ import print_function

import pandas as pd
import os 
import boto3
from datetime import datetime
ts = datetime.now().strftime("%Y-%m-%d")
    
root_folder = os.getcwd()
root_folder = "/Users/yutakahosoai/project/data/aerial"
print(root_folder)
image_root = '/Users/yutakahosoai/project/data/aerial/niihau/jpgs'

import boto3 
boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'hawaii-marine-debris'


/Users/yutakahosoai/project/data/aerial


In [7]:

s3_resource.Bucket(bucket_name).download_file("niihau/final_marine_debris_database_NI_UTM4N_boxes.geojson", "{}/final_marine_debris_database_NI_UTM4N_boxes.geojson".format(root_folder))
s3_resource.Bucket(bucket_name).download_file("niihau/niihau_tileindex.geojson", "{}/niihau_tileindex.geojson".format(root_folder))

# s3_resource.Bucket(bucket_name).download_file("niihau/annotations.csv", "{}/niihau/annotations.csv".format(root_folder))


In [11]:
s3_resource.Bucket(bucket_name).download_file("ml/v2/data/labels.csv", "{}/niihau/labels.csv".format(root_folder))
# s3://hawaii-marine-debris/

In [42]:
s3_resource.Bucket(bucket_name).download_file("niihau/deduped_annotations.csv", "{}/niihau/annotations.csv".format(root_folder))
labels_df = pd.read_csv("{}/niihau/labels.csv".format(root_folder), encoding='utf-8',names=['label','id'])
labels_df

,label,id
0,Bicycle,0
1,Boat,1
2,Surfboard,2
3,Bottle,3
4,Plastic bag,4
5,Wheel,5
6,Tire,6
7,Buoys,7
8,Cloth,8
9,Foam,9


In [62]:
annotations_df = pd.read_csv("{}/niihau/annotations.csv".format(root_folder), encoding='utf-8')

annotations_df['label'] = annotations_df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'wood','V':'Boat'})



for key, grouped in annotations_df[:10].groupby('s3_key'):
    print(key)
    print(len(grouped))
    if len(grouped)>1:
        print(type(grouped.ix[0]))

niihau/jpg/1_1920_04_07.jpg
1
niihau/jpg/1_1920_05_11.jpg
1
niihau/jpg/1_1920_06_12.jpg
1
niihau/jpg/1_1920_07_12.jpg
1
niihau/jpg/1_1920_07_13.jpg
2
<class 'pandas.core.frame.DataFrame'>
niihau/jpg/1_1920_08_13.jpg
4
<class 'pandas.core.frame.DataFrame'>


In [28]:
import random
def random_color():
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())
random_color()

'#3EB49C'

In [24]:
for i, v in labels_df.iterrows():
    print(v.label)
    break

Bicycle


In [35]:
d = {
    "tags_objects": ["urgent","risk","low_risk"],
    "tags_images": ["beach", "ocean", "inland"],
    "classes":[]
}

for i, v in labels_df.iterrows():
    color_dict = {"title":v.label,
                  'color':random_color(),
                  'shape':'rectangle'
                  }
    d['classes'].append(color_dict)
    
# {"classes": [{"title": "lemon", "color": "#9EDD7D", "shape": "bitmap"}, {"title": "kiwi", "color": "#A094D2", "shape": "bitmap"}], "tags_images": [], "tags_objects": []}
# import json
# print(json.dumps(d, indent=4))

with open('meta.json', 'w') as outfile:
    json.dump(d, outfile, indent=4)


{
    "tags_objects": [
        "urgent",
        "risk",
        "low_risk"
    ],
    "tags_images": [
        "beach",
        "ocean",
        "inland"
    ],
    "classes": [
        {
            "title": "Bicycle",
            "color": "#619A00",
            "shape": "rectangle"
        },
        {
            "title": "Boat",
            "color": "#A8A68A",
            "shape": "rectangle"
        },
        {
            "title": "Surfboard",
            "color": "#FA1D47",
            "shape": "rectangle"
        },
        {
            "title": "Bottle",
            "color": "#99B320",
            "shape": "rectangle"
        },
        {
            "title": "Plastic bag",
            "color": "#A3A083",
            "shape": "rectangle"
        },
        {
            "title": "Wheel",
            "color": "#3A5079",
            "shape": "rectangle"
        },
        {
            "title": "Tire",
            "color": "#82F390",
            "shape": "rectangle"
        

In [85]:
def get_image(s3_key):
    local_image_path = "{}/{}".format(image_root, os.path.basename(s3_key))
    if not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(s3_key, local_image_path)
    return local_image_path

from PIL import Image

def get_width_height(s3_key):
    img_path = get_image(s3_key)
    im = Image.open(img_path)
    width, height = im.size
    return width, height
width, height = get_width_height("niihau/jpg/1_1920_04_07.jpg")
print(width)

1000


In [ ]:


def generate_image_json(image_name, island, list_of_debris):
    width, height = get_width_height(image_name)
    return {
        "description": "{}_{}".format(island,image_name),
        "name": "{}_{}".format(island,image_name),
        "size": {
            "width": width,
            "height": height
        },
        "tags": ["debris"],
        "objects": list_of_debris
        }
def generate_obj_json(debris_row):
    return {
      "description": "",
      "tags": [],
      "bitmap": None,
      "classTitle": debris_row.label,
      "points": {
        "exterior": [
          [
            debris_row.x0,
            debris_row.y1
          ],
          [
            debris_row.x1,
            debris_row.y0
          ]
        ],
        "interior": []
      }
    }

for key, grouped in annotations_df[:5].groupby('s3_key'):
    list_of_objs = []
    for i, val in grouped.iterrows():
        list_of_objs.append(generate_obj_json(val))
    json_per_image = generate_image_json(key, 'niihau', list_of_objs)
#     print(json.dumps(json_per_image, indent=4))
    json_file_name = "{}/ann/{}".format(image_root, os.path.basename(key).replace(".jpg",".json"))
    with open(json_file_name, 'w') as outfile:
        json.dump(json_per_image, outfile, indent=4)
